# Lab 4 - Three ways for Agents to Interact

The 3 models:

1. Separate calls to Runner.run()

2. Agent Handoffs, turns within a Runner.run()

    A -> B -> C

3. Agents as Tools, turns within a Runner.run()

    A -> B -> A


Approach (1) is the least juicy & fashionable, but it's the one I recommend the most!

Approach (2) seems really hard to make reliable.

Approach (3) works OK with good enough prompts..

In [2]:
from agents import Agent, Runner, trace, handoff
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv(override=True)


True

In [3]:

entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
"""

evaluator_instructions = "You evaluate novel, exciting business ideas for Autonomous AI Agents"
evaluator_input = "Evaluate the following business idea; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents:\n\n"

## Now let's define 2 important Pydantic Objects to describe our Data

In [4]:
class Idea(BaseModel):
    title: str = Field(description="The title of the idea")
    description: str = Field(description="A detailed description of the idea, in Markdown")

    def print_summary(self):
        print(f"Title: {self.title}")
        display(Markdown(self.description))

In [5]:
class Evaluation(BaseModel):
    idea: Idea = Field(description="The idea that you are evaluating")
    uniqueness_feedback: str = Field(description="Your view on how unique the idea is")
    uniqueness_score: int = Field(description="How unique the idea is, on a scale of 1 to 10, where 1 is mundane and 10 is groundbreaking")
    commercial_feedback: str = Field(description="Your view on the commercial potential of the idea")
    commercial_score: int = Field(description="How high the commercial potential of the idea is from 1 to 10, where 1 is barely profitable and 10 is a billion dollar idea")
    autonomy_feedback: str = Field(description="Your commentary on to what extent the idea truly benefits from the autonomous nature of AI Agents")
    autonomy_score: int = Field(description="How deeply the idea involves autonomous agents from 1 to 10, where 1 is that it plays no meaningful role, and 10 is that it is pivotal")

    def is_brilliant(self):
        return self.uniqueness_score > 8 and self.commercial_score > 8 and self.autonomy_score > 8
    
    def print_summary(self):
        print(f"Idea: {self.idea.title}")
        print(f"Uniqueness: {self.uniqueness_score}/10: ({self.uniqueness_feedback})")
        print(f"Commercial: {self.commercial_score}/10: ({self.commercial_feedback})")
        print(f"Autonomy: {self.autonomy_score}/10: ({self.autonomy_feedback})")

## Approach 1 (Highly Recommended!) - Call Runner.run() separately

In [6]:
entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea)
result = await Runner.run(entrepreneur, entrepreneur_input)
idea = result.final_output_as(Idea)
idea.print_summary()

Title: Autonomous AI Real-Time Supply Chain Orchestrator


### Industry: Logistics and Supply Chain Management

### Concept Overview
Develop an Autonomous AI Agent that acts as a real-time Supply Chain Orchestrator, capable of independently managing and optimizing complex supply chains across global networks. This AI agent would monitor shipments, production schedules, and inventory levels continuously, autonomously responding to disruptions or inefficiencies by recalculating logistics routes, adjusting orders, and negotiating with suppliers and carriers using integrated smart contracts.

### Unique Selling Points
- **True Autonomy:** Unlike existing supply chain software that depends on human input for adjustments, this agent autonomously identifies bottlenecks or delays (e.g., due to weather, geopolitical issues, or demand spikes) and implements corrective actions without requiring human intervention.
- **Integrated Negotiations:** Equipped with NLP and contract negotiation capabilities, the AI can dynamically renegotiate terms or procure alternative suppliers and logistics vendors on behalf of the business.
- **End-to-End Visibility:** Combines data from IoT sensors, transportation trackers, production lines, and demand forecasting AI to create a holistic, up-to-the-minute picture and controls the supply chain accordingly.
- **Scalable Impact:** Suitable for multinational corporations managing thousands of SKU lines, this system can reduce lead times, cut inventory costs, and improve responsiveness to market changes.

### Commercial Potential
Global supply chains are increasingly complex and vulnerable to disruption. Autonomous AI supply chain orchestrators could drastically reduce operational costs and losses due to delays or mismanagement, offering a competitive edge to adopters. The system could be sold as a subscription-based SaaS model or integrated consultancy solution, targeting industries such as automotive, retail, pharmaceuticals, and electronics.

---

This idea leverages autonomous AI agent capabilities to independently sense, plan, and act within a high-stakes, dynamic environment, delivering measurable value by optimizing processes that currently require extensive human oversight.

In [7]:
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)
result = await Runner.run(evaluator, f"{evaluator_input}{idea}")
evaluation = result.final_output_as(Evaluation)
evaluation.print_summary()

Idea: Autonomous AI Real-Time Supply Chain Orchestrator
Uniqueness: 8/10: (The idea is quite unique in its deep level of autonomy and integration. While there are many supply chain management tools and AI-powered analytics, few—if any—offer full autonomous orchestration with real-time adjustments, negotiation capabilities, and seamless integration of diverse data streams using smart contracts. The combination of these features into a single autonomous agent that can operate with minimal human intervention represents a significant step beyond current market offerings.)
Commercial: 9/10: (The commercial potential is very high. Supply chain disruptions cost global industries billions, and a system that can autonomously minimize these losses and optimize operations can offer tremendous ROI. The scalability to large multinational corporations and industries with complex supply chains enhances its market size. The SaaS or consultancy model is practical and aligns well with current enterprise

In [8]:
entrepreneur_latest = f"You were originally given the following task:\n\n{entrepreneur_input}\n\n"
remaining_attempts = 4

In [9]:
with trace("Entrepreneur Approach 1"):
    while not evaluation.is_brilliant() and remaining_attempts:
        remaining_attempts -= 1
        entrepreneur_latest += f"You responded with this idea:\n\n{idea}\n\nYou received this feedback:\n\n{evaluation}\n\n"
        entrepreneur_latest += "Now respond with an improved idea that adresses the feedback. Do not directly reference the feedback.\n\n"
        result = await Runner.run(entrepreneur, entrepreneur_latest)
        idea = result.final_output_as(Idea)
        idea.print_summary()
        result = await Runner.run(evaluator, f"{evaluator_input}{idea}")
        evaluation = result.final_output_as(Evaluation)
        evaluation.print_summary()

Title: Autonomous AI Global Supply Chain Resilience Hub


### Industry: Logistics and Supply Chain Management

### Concept Overview
Create an advanced Autonomous AI Agent platform, the Global Supply Chain Resilience Hub, designed to provide fully autonomous, end-to-end supply chain orchestration with a focus on proactive disruption anticipation and recovery. This agent continuously ingests and analyzes heterogeneous data streams — including IoT device inputs, satellite and weather data, geopolitical news, trade regulations, and market demand signals — to autonomously forecast risks and initiate mitigation strategies in real time.

### Key Innovations
- **Predictive Disruption Management:** Uses deep learning to anticipate multi-modal risks such as natural disasters, customs delays, or supplier insolvencies days or weeks ahead, allowing the agent to preemptively reroute logistics, adjust production, or secure alternative sourcing.
- **Autonomous Adaptive Contracting:** Employs natural language understanding and negotiation frameworks to autonomously manage and renegotiate smart contracts with suppliers, carriers, and partners, dynamically optimizing terms based on evolving circumstances.
- **Federated Multi-Partner Integration:** Supports secure, privacy-preserving collaboration across diverse stakeholders (manufacturers, 3PLs, distributors) through federated learning models, enabling ecosystem-wide synchronized decision making without exposing sensitive data.
- **Dynamic Inventory and Capacity Optimization:** Continuously rebalances inventory and capacity allocations across global warehouses and production facilities in reaction to live demand and supply fluctuations.

### Commercial Potential
The solution addresses the high cost and complexity of global supply chain disruptions by offering companies a resilient, self-healing operations platform that minimizes downtime and reduces excess inventory holding. Delivered as a modular SaaS suite tailored by industry vertical, this platform targets global enterprises in automotive, electronics, pharmaceuticals, and consumer goods. Early adopters gain significant competitive advantages through improved service levels, lowered costs, and increased supply chain agility.

---

This idea moves beyond current supply chain software by embedding deep autonomous intelligence that anticipates, negotiates, and adapts across complex global networks in real time, offering a transformative leap in supply chain resilience and efficiency.

Idea: Autonomous AI Global Supply Chain Resilience Hub
Uniqueness: 9/10: (The idea is quite unique in the realm of autonomous AI agents applied to supply chain management. While predictive analytics and some automation exist in current supply chain solutions, the integration of fully autonomous, end-to-end orchestration combining multi-source real-time data ingestion, proactive risk forecasting, autonomous contract renegotiation, and federated learning for ecosystem-wide collaboration is a novel and comprehensive approach. Especially the autonomous adaptive contracting and federated multi-partner integration are standout innovations not commonly seen together in existing platforms.)
Commercial: 9/10: (The commercial potential is substantial, given the critical importance and fragility of global supply chains worldwide. Enterprises across multiple major industries face significant losses from disruptions and inefficiencies, so a platform that can proactively manage risks, autonomously a

## Approach 2 - Handoff

This seems to be the more unpredictable of the approaches, and I struggle to get it to ever handoff more than 1 loop..

In [10]:
entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents that you pass to the Evaluator to evaluate"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
You MUST handoff your idea to the Evaluator to be evaluated by using the tool Handoff_to_Evaluator. You must always call this tool with your idea or an improved idea.
"""

evaluator_instructions = """
You evaluate novel, exciting business ideas for Autonomous AI Agents.
Evaluate the business idea that you are given; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents.
If you would not score the idea 9/10 or 10/10 in all 3 categories, then you must handoff the idea back to the Entrepreneur to improve it by using the tool Handoff_to_Entrepreneur and providing your feedback.
if you do score the idea 9/10 or better in all categories, then respond with the evaluation.
"""

In [11]:
entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea)
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)


def on_handoff(ctx, input):
    print("Handoff called")

entrepreneur.handoffs = [handoff(
    agent=evaluator,
    input_type=Idea,
    on_handoff=on_handoff,
    tool_name_override="Handoff_to_Evaluator",
    tool_description_override="Use this tool to handoff your idea to the Evaluator to evaluate. You must always call this tool."
)]

evaluator.handoffs = [handoff(
    agent=entrepreneur,
    input_type=Evaluation,
    on_handoff=on_handoff,
    tool_name_override="Handoff_to_Entrepreneur",
    tool_description_override="Use this tool to handoff your feedback to the Entrepreneur to improve the idea. You should call this tool if your feedback is less than 9/10 in any category."
)] 

In [12]:
with trace("Entrepreneur Approach 2"):
    result = await Runner.run(entrepreneur, entrepreneur_input, max_turns=10)

Handoff called
Handoff called


In [13]:
result.final_output.print_summary()

Title: AI-Driven Autonomous Facility Management Agents


### Industry: Commercial Real Estate and Facility Management

### Concept:
Develop autonomous AI Agents specialized in facility management that independently monitor, predict, and manage building systems to optimize energy use, maintenance schedules, security, and occupant comfort without human intervention.

### How It Works:
- **Continuous Autonomous Monitoring:** AI agents utilize IoT sensors and building management systems data to continuously monitor HVAC, lighting, security cameras, elevators, and other building operations.
- **Predictive Maintenance:** Agents analyze patterns and predict equipment failures or inefficiencies, scheduling maintenance autonomously before issues arise, reducing downtime and repair costs.
- **Energy Optimization:** Autonomously adjust heating, cooling, lighting, and energy consumption patterns in real-time based on occupancy patterns and weather forecasts, significantly lowering energy costs.
- **Security and Access Control:** Agents independently identify security threats or unsafe conditions by integrating data from cameras and sensors, alerting authorities or locking down zones as needed.
- **Occupant Comfort Management:** Dynamically adjust lighting, temperature, and air quality to maximize occupant well-being using feedback loops from autonomous sensing and user preferences.

### Commercial Impact:
- Significant cost savings from reduced energy consumption and maintenance expenses.
- Enhanced building safety and security through proactive risk management.
- Increased tenant satisfaction and retention due to improved comfort and reliability.
- Scalable to manage multiple properties or large commercial complexes autonomously, offering a high-impact service for facility management companies or real estate owners.

### Why Autonomous AI Agents?
The fully autonomous nature eliminates the constant need for human monitoring and intervention, enabling 24/7 proactive building management that can rapidly adapt to changing conditions at scale. This disrupts traditional facility management approaches dependent on scheduled checks and reactive repairs, providing a transformative leap in efficiency and service quality.

## Approach 3 - Agents as Tools

In [14]:

entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
You must call the Evaluator tool to Evaluate the idea. If the idea is not 9 or 10 in all 3 scores,
then you must improve the idea and call the Evaluator tool again.
Keep improving the idea and calling the Evaluator tool until you get a 9 or 10 in all 3 scores.
Then finally respond with the idea.
"""
evaluator_instructions = """
You evaluate novel, exciting business ideas for Autonomous AI Agents.
Evaluate the following business idea; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents.
"""



In [15]:
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)
evaluator_tool = evaluator.as_tool(
    tool_name="Evaluator_Idea_Evaluator",
    tool_description="Evaluates the idea for uniqueness, potential commercial impact, and use of Autonomous AI Agents",
)

entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea, tools=[evaluator_tool])

In [16]:
with trace("Entrepreneur Approach 3"):
    result = await Runner.run(entrepreneur, entrepreneur_input)
    idea = result.final_output_as(Idea)
    idea.print_summary()

Title: Autonomous Cooperative AgriSentinel AI Network with Integrated Autonomous Financial Services


Enhance the existing Autonomous Cooperative AgriSentinel AI Network by integrating autonomous AI-driven financial services. This integration includes real-time dynamic pricing adjustments, automated insurance claims processing through AI agents analyzing crop damage via drone surveillance, and autonomous investment recommendations customized based on each farm's performance and current market trends. The financial AI agents operate in seamless coordination with operational farming agents, creating a comprehensive, fully autonomous agricultural management ecosystem encompassing farming, logistics, compliance, and finance without human intervention, thereby maximizing efficiency and profitability.

This platform uniquely combines autonomous crop and resource management across cooperative farm networks with advanced financial services, enabling farms to optimize operations, reduce costs, ensure regulatory compliance, and enhance financial outcomes simultaneously. The depth of autonomy across these multiple domains represents a breakthrough in smart agriculture, offering significant commercial potential globally in the growing agri-tech sector.

## OpenAI Traces

We've been writing all this information to OpenAI traces, the 'observability' framework built into OpenAI.

You can take a look at these traces here:

https://platform.openai.com/traces

There's also a nice way to visualize:

In [ ]:
from agents.extensions.visualization import draw_graph
draw_graph(entrepreneur)
